In [0]:
import pandas as pd
import numpy as np

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
data = pd.read_csv("/content/drive/My Drive/Colab Notebooks/covid/DCIS_POPRES1_25052020170201288.csv")

In [0]:
df = data.loc[(data.SEXISTAT1 == 9) &
                (data.STATCIV2 == 99) &
                (data.ITTER107.str.len() >= 5) &
                (data.ETA1 != "TOTAL")][["Territorio", "ETA1", "Value"]]

In [0]:
df.ETA1 = df.ETA1.str.replace("Y","")
df.ETA1 = df.ETA1.str.replace("_GE","")
df.Territorio = df.Territorio.str.replace("Valle d'Aosta / Vallée d'Aoste",
                                                "Valle d'Aosta")
df.Territorio = df.Territorio.str.replace("Bolzano / Bozen", "Bolzano")
df = df.astype({"ETA1": int})

In [0]:
df_final = pd.DataFrame()

for provincia in df.Territorio.unique():
  df1 = df.loc[df["Territorio"] == provincia]

  a = df1[df1["ETA1"] <= 25].groupby(["Territorio"]).agg({"Value" : sum})
  b = df1[(df1["ETA1"] > 25) & (df1["ETA1"] <= 50)].groupby(["Territorio"]).agg({"Value" : sum})
  c = df1[(df1["ETA1"] > 50) & (df1["ETA1"] <= 75)].groupby(["Territorio"]).agg({"Value" : sum})
  d = df1[df1["ETA1"] > 75].groupby(["Territorio"]).agg({"Value" : sum})

  tmp = pd.concat([a, b, c, d])
  tmp.reset_index(level=0, inplace=True)
  tmp = tmp.append(pd.DataFrame([{"Territorio" : provincia, "Value" : tmp.Value.sum()}]), ignore_index=True)
  tmp["Eta"] = ["0-25", "25-50", "50-75", "75-100", "Total"]
  tmp["Percentage"] = tmp.Value.apply(lambda x: x / tmp.Value.values[-1])

  df_final = pd.concat([df_final, tmp])

In [0]:
df_final.to_csv("/content/drive/My Drive/Colab Notebooks/covid/pop_prov_age.csv", index=False)